In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
from scipy import stats
from sklearn.preprocessing import LabelEncoder

# 1. Carregar e preparar os dados
def load_data(filepath):
    df = pd.read_csv(filepath, parse_dates=['Unnamed: 0'], index_col='Unnamed: 0')
    df.rename(columns={'setor': 'sector'}, inplace=True)
    df = df.sort_index()
    df = remover_colunas_com_nans_totais(df)
    return df

def remover_colunas_com_nans_totais(df):
    colunas_para_remover = [
        "1. open", "2. high", "3. low", "4. close",
        "5. adjusted close", "6. volume",
        "7. dividend amount", "8. split coefficient"
    ]

    colunas_existentes = [col for col in colunas_para_remover if col in df.columns]

    if colunas_existentes:
        print(f"Removendo colunas com muitos NaNs: {colunas_existentes}")
        df = df.drop(columns=colunas_existentes)
    else:
        print("Nenhuma das colunas especificadas foi encontrada.")

    return df


def print_nan_por_feature(df):
    nan_por_coluna = df.isna().sum()
    nan_por_coluna = nan_por_coluna[nan_por_coluna > 0]  # Mostra apenas colunas com pelo menos um NaN

    if nan_por_coluna.empty:
        print("Nenhuma feature contém valores NaN.")
    else:
        print("Quantidade de NaN por feature:")
        for coluna, qtd in nan_por_coluna.items():
            print(f"{coluna}: {qtd} NaNs")


# 2. Feature Engineering sem TA-Lib
def create_features(df):
    # Codificar setor como variável categórica
    le = LabelEncoder()
    df['sector_encoded'] = le.fit_transform(df['sector'])
    
    # Retornos
    df['retorno_1D'] = df.groupby('ticker')['close'].pct_change()
    for window in [5, 21, 63, 252]:  # 1 semana, 1 mês, 3 meses, 1 ano
        df[f'retorno_{window}D'] = df.groupby('ticker')['close'].pct_change(window)
    
    # Volatilidade
    df['volatilidade_21D'] = df.groupby('ticker')['retorno_1D'].rolling(21).std().values
    
    # Volume
    df['volume_avg_21D'] = df.groupby('ticker')['volume'].rolling(21).mean().values
    df['volume_avg_21D'] = df['volume_avg_21D'].replace(0, 1)
    df['volume_spike'] = df['volume'] / df['volume_avg_21D']
    
    # Médias Móveis (substituindo TA-Lib)
    df['SMA_50'] = df.groupby('ticker')['close'].rolling(50).mean().values
    df['SMA_200'] = df.groupby('ticker')['close'].rolling(200).mean().values
    
    # RSI (implementação manual simplificada)
    def calculate_rsi(series, window=14):
        delta = series.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(window).mean()
        avg_loss = loss.rolling(window).mean()
        
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    
    df['RSI_14'] = df.groupby('ticker')['close'].transform(calculate_rsi)
    
    # Bollinger Bands (implementação manual)
    def calculate_bollinger_bands(series, window=20, num_std=2):
        sma = series.rolling(window).mean()
        std = series.rolling(window).std()
        upper = sma + (std * num_std)
        lower = sma - (std * num_std)
        return upper, lower
    
    df['BB_upper'], df['BB_lower'] = calculate_bollinger_bands(df['close'])
    
    # Dividendos
    df['dividend_yield'] = df['dividend_amount'] / df['close']
    df['dividend_payment'] = (df['dividend_amount'] > 0).astype(int)
    
    # Features agregadas por setor
    df['sector_avg_retorno_21D'] = df.groupby(['sector', df.index])['retorno_21D'].transform('mean')
    df['sector_avg_volume'] = df.groupby(['sector', df.index])['volume'].transform('mean')
    
    # Target: Retorno em 1 ano (252 dias úteis)
    df['target'] = df.groupby('ticker')['adjusted_close'].transform(
        lambda x: x.shift(-252) / x - 1
    )
    
    print_nan_por_feature(df)

    return df.dropna()

# 3. Tratamento de Outliers
def remove_outliers(df, z_threshold=3):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in ['target', 'split_coefficient', 'sector_encoded']]

    # Calcula z-score e alinha com os índices do DataFrame
    z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))
    z_scores_df = pd.DataFrame(z_scores, index=df.index, columns=numeric_cols)

    # Diagnóstico
    for col in numeric_cols:
        pct_removidas = (z_scores_df[col] > z_threshold).mean() * 100
        # print(f"{col}: {pct_removidas:.2f}% das linhas acima do z={z_threshold}")

    # Máscara global: apenas linhas com z-score aceitável em todas as colunas
    mask = (z_scores_df < z_threshold).all(axis=1)
    df = df[mask]

    # Volume mínimo e remoção de NaNs
    df = df[df['volume'] > 10000]
    return df.dropna()

def check_nan_inf(df):
    numeric_df = df.select_dtypes(include=[np.number])  # apenas colunas numéricas
    result = pd.DataFrame(index=df.columns)

    result['NaN'] = df.isna().sum()
    result['inf'] = numeric_df.applymap(np.isposinf).sum()
    result['-inf'] = numeric_df.applymap(np.isneginf).sum()
    result['Total (inf ou NaN)'] = result[['NaN', 'inf', '-inf']].sum(axis=1)

    return result[result['Total (inf ou NaN)'] > 0].sort_values('Total (inf ou NaN)', ascending=False)

# 4. Treinamento do Modelo
def train_model(df):
    # Features e target
    features = ['retorno_1D', 'retorno_5D', 'retorno_21D', 'retorno_63D',
                'volatilidade_21D', 'volume_avg_21D', 'volume_spike',
                'RSI_14', 'SMA_50', 'SMA_200', 'BB_upper', 'BB_lower',
                'dividend_yield', 'dividend_payment',
                'sector_encoded', 'sector_avg_retorno_21D', 'sector_avg_volume']
    
    X = df[features]
    y = df['target']
    
    # Cross-validation temporal
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Configuração do modelo
    params = {
        'objective': 'mae',
        'num_iterations': 1000,
        'learning_rate': 0.01,
        'max_depth': 7,
        'num_leaves': 31,
        'random_state': 42,
        'verbosity': -1
    }
    
    metrics = []
    feature_importances = pd.DataFrame()
    models = []  # Para armazenar todos os modelos
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[valid_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=100)
            ]
        )
        models.append(model)  # Armazena o modelo
        
        # Avaliação
        preds = model.predict(X_test)
        metrics.append({
            'Fold': fold + 1,
            'MAE': mean_absolute_error(y_test, preds),
            'RMSE': np.sqrt(mean_squared_error(y_test, preds)),
            'R2': r2_score(y_test, preds)
        })
        
        # Importância das features
        fold_importance = pd.DataFrame({
            'feature': features,
            'importance': model.feature_importance(importance_type='gain'),
            'fold': fold + 1
        })
        feature_importances = pd.concat([feature_importances, fold_importance])
    
    # Métricas médias
    avg_metrics = pd.DataFrame(metrics).mean()
    print("\nMétricas médias de validação:")
    print(f"MAE: {avg_metrics['MAE']:.4f}")
    print(f"RMSE: {avg_metrics['RMSE']:.4f}")
    print(f"R²: {avg_metrics['R2']:.4f}")
    
    # Analisar importância das features
    mean_importance = feature_importances.groupby('feature')['importance'].mean().sort_values(ascending=False)
    print("\nImportância das features:")
    print(mean_importance.head(15))
    
    return models[-1], features, pd.DataFrame(metrics)  # Retorna o último modelo treinado

# Pipeline completo ajustado
if __name__ == "__main__":
    # Carregar dados
    df = load_data('datasets/acoes_consolidadas_todas.csv')
    
    # Criar features
    df = create_features(df)
    print(f"Total de registros após feature engineering: {len(df)}")
    print(f"Setores disponíveis: {df['sector'].unique()}")

    # Remover outliers
    df = remove_outliers(df)
    print(f"Total de registros após remoção de outliers: {len(df)}")
    
    # Treinar modelo
    model, features, metrics_df = train_model(df)
    
    # Salvar modelo e resultados
    model.save_model('modelo_lgbm_sem_talib.txt')  # Método direto do Booster
    metrics_df.to_csv('metricas_validacao.csv', index=False)
    pd.Series(features).to_csv('features_utilizadas.csv', index=False)
    
    print("\nModelo treinado com sucesso sem TA-Lib!")
    print("Arquivos gerados:")
    print("- modelo_lgbm_sem_talib.txt")
    print("- metricas_validacao.csv")
    print("- features_utilizadas.csv")

Removendo colunas com muitos NaNs: ['1. open', '2. high', '3. low', '4. close', '5. adjusted close', '6. volume', '7. dividend amount', '8. split coefficient']


C:\Users\Colaborador\AppData\Local\Temp\ipykernel_2752\926823840.py:54: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['retorno_1D'] = df.groupby('ticker')['close'].pct_change()
C:\Users\Colaborador\AppData\Local\Temp\ipykernel_2752\926823840.py:56: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'retorno_{window}D'] = df.groupby('ticker')['close'].pct_change(window)
C:\Users\Colaborador\AppData\Local\Temp\ipykernel_2752\926823840.py:56: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either f

Quantidade de NaN por feature:
open: 5082 NaNs
high: 5082 NaNs
low: 5082 NaNs
close: 5082 NaNs
adjusted_close: 5082 NaNs
volume: 5082 NaNs
dividend_amount: 5082 NaNs
split_coefficient: 5082 NaNs
retorno_1D: 5247 NaNs
retorno_5D: 5907 NaNs
retorno_21D: 8547 NaNs
retorno_63D: 15466 NaNs
retorno_252D: 46462 NaNs
volatilidade_21D: 8547 NaNs
volume_avg_21D: 8382 NaNs
volume_spike: 13375 NaNs
SMA_50: 13167 NaNs
SMA_200: 37770 NaNs
RSI_14: 9010 NaNs
BB_upper: 75326 NaNs
BB_lower: 75326 NaNs
dividend_yield: 5082 NaNs
sector_avg_retorno_21D: 3228 NaNs
sector_avg_volume: 1364 NaNs
target: 46462 NaNs
Total de registros após feature engineering: 283528
Setores disponíveis: ['Alimentos' 'Saúde' 'Telecomunicações' 'Agropecuária' 'Indústria Química'
 'Logística' 'Educação' 'Varejo' 'Industrial' 'Transportes'
 'Construção Civil' 'Financeiro' 'Energia' 'Químico' 'Serviços' 'Consumo'
 'Siderurgia' 'Imobiliário' 'Saneamento' 'Holdings' 'Tecnologia'
 'Petróleo' 'Mineração' 'Papel e Celulose' 'Distribuição

In [9]:
prints

['open: 0.00% das linhas acima do z=3',
 'high: 0.00% das linhas acima do z=3',
 'low: 0.00% das linhas acima do z=3',
 'close: 0.00% das linhas acima do z=3',
 'adjusted_close: 0.00% das linhas acima do z=3',
 'volume: 1.59% das linhas acima do z=3',
 'dividend_amount: 0.48% das linhas acima do z=3',
 'retorno_1D: 0.00% das linhas acima do z=3',
 'retorno_5D: 0.00% das linhas acima do z=3',
 'retorno_21D: 0.00% das linhas acima do z=3',
 'retorno_63D: 0.00% das linhas acima do z=3',
 'retorno_252D: 0.00% das linhas acima do z=3',
 'volatilidade_21D: 0.03% das linhas acima do z=3',
 'volume_avg_21D: 1.75% das linhas acima do z=3',
 'volume_spike: 0.00% das linhas acima do z=3',
 'SMA_50: 0.07% das linhas acima do z=3',
 'SMA_200: 0.27% das linhas acima do z=3',
 'RSI_14: 0.00% das linhas acima do z=3',
 'BB_upper: 0.03% das linhas acima do z=3',
 'BB_lower: 0.03% das linhas acima do z=3',
 'dividend_yield: 0.16% das linhas acima do z=3',
 'dividend_payment: 1.07% das linhas acima do z=